### Analytics Vidhya: 52.5823376761

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)  
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline 
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/train.csv')
center=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/fulfilment_center_info.csv')
meal=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/meal_info.csv')
test=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/test.csv')

##### Data Pre-Processing

In [3]:
print("The Shape of Demand dataset :",data.shape)
print("The Shape of Fulmilment Center Information dataset :",center.shape)
print("The Shape of Meal information dataset :",meal.shape)
print("The Shape of Test dataset :",test.shape)

The Shape of Demand dataset : (456548, 9)
The Shape of Fulmilment Center Information dataset : (77, 5)
The Shape of Meal information dataset : (51, 3)
The Shape of Test dataset : (32573, 8)


In [4]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [5]:
test['num_orders']=123456 ### Assigning random number for Target Variable of Test Data.

In [6]:
test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1028232,146,55,1885,158.11,159.11,0,0,123456
1,1127204,146,55,1993,160.11,159.11,0,0,123456
2,1212707,146,55,2539,157.14,159.14,0,0,123456
3,1082698,146,55,2631,162.02,162.02,0,0,123456
4,1400926,146,55,1248,163.93,163.93,0,0,123456


In [7]:
center.head()

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [8]:
meal.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [9]:
data=pd.concat([data,test],axis=0)

In [10]:
data=data.merge(center,on='center_id',how='left')

In [11]:
data=data.merge(meal,on='meal_id',how='left')

In [12]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian


In [13]:
data.isnull().sum()

id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
city_code                0
region_code              0
center_type              0
op_area                  0
category                 0
cuisine                  0
dtype: int64

##### Deriving New Features

In [14]:
#Discount Amount
data['discount amount']=data['base_price']-data['checkout_price']

In [15]:
#Discount Percent
data['discount percent'] = ((data['base_price']-data['checkout_price'])/data['base_price'])*100

In [16]:
#Discount Y/N
data['discount y/n'] = [1 if x>0 else 0 for x in (data['base_price']-data['checkout_price'])]

In [17]:
data=data.sort_values(['center_id', 'meal_id', 'week']).reset_index()

In [18]:
#Compare Week Price
data['compare_week_price'] = data['checkout_price'] - data['checkout_price'].shift(1) 

In [19]:
data['compare_week_price'][data['week']==1]=0

In [20]:
data=data.sort_values(by='index').reset_index().drop(['level_0','index'],axis=1)

In [21]:
# Compare Week Price Y/N
data['compare_week_price y/n'] = [1 if x>0 else 0 for x in data['compare_week_price']]

In [22]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai,15.46,10.151684,1,0.0,0
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai,-1.00,-0.736214,0,0.0,0
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai,1.00,0.736052,1,0.0,0
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian,98.03,22.405321,1,0.0,0
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian,-1.00,-0.412371,0,0.0,0


In [23]:
data.isnull().sum()

id                        0
week                      0
center_id                 0
meal_id                   0
checkout_price            0
base_price                0
emailer_for_promotion     0
homepage_featured         0
num_orders                0
city_code                 0
region_code               0
center_type               0
op_area                   0
category                  0
cuisine                   0
discount amount           0
discount percent          0
discount y/n              0
compare_week_price        0
compare_week_price y/n    0
dtype: int64

##### Train Test Split

In [24]:
train=data[data['week'].isin(range(1,146))]
test=data[data['week'].isin(range(146,156))]

In [25]:
print("The Shape of Train dataset :",train.shape)
print("The Shape of Test dataset :",test.shape)

The Shape of Train dataset : (456548, 20)
The Shape of Test dataset : (32573, 20)


Also when we checked the number of orders with respect to City, we could see that City - 590 has the highest number of orders - 18.5M orders which is almost 10M orders higher than the City with second highest number of orders - City 526 - 8.6M orders.

##### Encoding City 

As per our observation from our barchart of the City against the number of orders. There the high significant difference between the Top 3 cities which has the highest number of orders. Therefore, in our first approach we will encode the City with Highest No. of Orders as CH1, City with 2nd Highest No. of Orders as CH2 and City with 3rd Highest No. of Orders as CH3 and rest all of the cities which does not have much significant differences between the number of orders as CH4.

In [26]:
city4={590:'CH1', 526:'CH2', 638:'CH3'}
data['city_enc_4']=data['city_code'].map(city4)
data['city_enc_4']=data['city_enc_4'].fillna('CH4')

In [27]:
data['city_enc_4'].value_counts()

CH4    362341
CH1     58708
CH2     46589
CH3     21483
Name: city_enc_4, dtype: int64

In [28]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n,city_enc_4
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai,15.46,10.151684,1,0.0,0,CH4
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai,-1.00,-0.736214,0,0.0,0,CH4
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai,1.00,0.736052,1,0.0,0,CH4
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian,98.03,22.405321,1,0.0,0,CH4
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian,-1.00,-0.412371,0,0.0,0,CH4


In [29]:
data.isnull().sum()

id                        0
week                      0
center_id                 0
meal_id                   0
checkout_price            0
base_price                0
emailer_for_promotion     0
homepage_featured         0
num_orders                0
city_code                 0
region_code               0
center_type               0
op_area                   0
category                  0
cuisine                   0
discount amount           0
discount percent          0
discount y/n              0
compare_week_price        0
compare_week_price y/n    0
city_enc_4                0
dtype: int64

##### Copying to New DataFrame

In [30]:
datax=data.copy()
datax.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n,city_enc_4
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai,15.46,10.151684,1,0.0,0,CH4
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai,-1.00,-0.736214,0,0.0,0,CH4
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai,1.00,0.736052,1,0.0,0,CH4
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian,98.03,22.405321,1,0.0,0,CH4
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian,-1.00,-0.412371,0,0.0,0,CH4


##### Enoding all the Categorical Features

In [31]:
datax['center_id']=datax['center_id'].astype('object')
datax['meal_id']=datax['meal_id'].astype('object')
datax['region_code']=datax['region_code'].astype('object')

In [32]:
obj=datax[['center_id','meal_id','region_code','center_type','category','cuisine','city_enc_4']]
num=datax.drop(['center_id','meal_id','region_code','center_type','category','cuisine','city_enc_4'],axis=1)

In [33]:
encode1=pd.get_dummies(obj,drop_first = True)

In [34]:
datax=pd.concat([num,encode1],axis=1)

In [35]:
datax.head()

,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,op_area,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n,center_id_11,center_id_13,center_id_14,center_id_17,center_id_20,center_id_23,center_id_24,center_id_26,center_id_27,center_id_29,center_id_30,center_id_32,center_id_34,center_id_36,center_id_39,center_id_41,center_id_42,center_id_43,center_id_50,center_id_51,center_id_52,center_id_53,center_id_55,center_id_57,center_id_58,center_id_59,center_id_61,center_id_64,center_id_65,center_id_66,center_id_67,center_id_68,center_id_72,center_id_73,center_id_74,center_id_75,center_id_76,center_id_77,center_id_80,center_id_81,center_id_83,center_id_86,center_id_88,center_id_89,center_id_91,center_id_92,center_id_93,center_id_94,center_id_97,center_id_99,center_id_101,center_id_102,center_id_104,center_id_106,center_id_108,center_id_109,center_id_110,center_id_113,center_id_124,center_id_126,center_id_129,center_id_132,center_id_137,center_id_139,center_id_143,center_id_145,center_id_146,center_id_149,center_id_152,center_id_153,center_id_157,center_id_161,center_id_162,center_id_174,center_id_177,center_id_186,meal_id_1109,meal_id_1198,meal_id_1207,meal_id_1216,meal_id_1230,meal_id_1247,meal_id_1248,meal_id_1311,meal_id_1438,meal_id_1445,meal_id_1525,meal_id_1543,meal_id_1558,meal_id_1571,meal_id_1727,meal_id_1754,meal_id_1770,meal_id_1778,meal_id_1803,meal_id_1847,meal_id_1878,meal_id_1885,meal_id_1902,meal_id_1962,meal_id_1971,meal_id_1993,meal_id_2104,meal_id_2126,meal_id_2139,meal_id_2290,meal_id_2304,meal_id_2306,meal_id_2322,meal_id_2444,meal_id_2490,meal_id_2492,meal_id_2494,meal_id_2539,meal_id_2569,meal_id_2577,meal_id_2581,meal_id_2631,meal_id_2640,meal_id_2664,meal_id_2704,meal_id_2707,meal_id_2760,meal_id_2826,meal_id_2867,meal_id_2956,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai,city_enc_4_CH2,city_enc_4_CH3,city_enc_4_CH4
0,1379560,1,136.83,152.29,0,0,177,647,2.0,15.46,10.151684,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,1466964,1,136.83,135.83,0,0,270,647,2.0,-1.00,-0.736214,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,1346989,1,134.86,135.86,0,0,189,647,2.0,1.00,0.736052,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1338232,1,339.50,437.53,0,0,54,647,2.0,98.03,22.405321,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,1448490,1,243.50,242.50,0,0,40,647,2.0,-1.00,-0.412371,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [36]:
abs(train.corr()['num_orders']).sort_values(ascending=False)

num_orders                1.000000
homepage_featured         0.294490
checkout_price            0.282108
emailer_for_promotion     0.277147
base_price                0.222306
discount percent          0.205188
op_area                   0.176976
discount amount           0.152950
compare_week_price        0.137849
discount y/n              0.111766
center_id                 0.053035
city_code                 0.041596
region_code               0.029744
compare_week_price y/n    0.022898
week                      0.017210
meal_id                   0.010597
id                        0.000702
Name: num_orders, dtype: float64

##### Base Model

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [38]:
sc=StandardScaler()

cat=datax.drop(['checkout_price','base_price','discount amount','discount percent','compare_week_price'],axis=1)
num=datax[['checkout_price','base_price','discount amount','discount percent','compare_week_price']]
scal= pd.DataFrame(sc.fit_transform(num),columns=num.columns)
datas=pd.concat([scal,cat],axis=1)

In [39]:
datay=datas.copy()

In [40]:
datay['Quarter']=(datas['week']/13).astype('int64')

In [41]:
datay['Quarter'] = datay['Quarter'].map({0:'Q1',
                         1:'Q2',
                         2:'Q3',
                         3:'Q4',
                         4:'Q1',
                         5:'Q2',
                         6:'Q3',
                         7:'Q4',
                         8:'Q1',
                         9:'Q2',
                         10:'Q3',
                         11:'Q4'})

In [42]:
datay['Quarter'].value_counts()

Q4    124839
Q3    124225
Q2    122386
Q1    117671
Name: Quarter, dtype: int64

In [43]:
datay['Year']=(datas['week']/52).astype('int64')

In [44]:
datay['Year'] = datay['Year'].map({0:'Y1',
                         1:'Y2',
                         2:'Y3'})

In [45]:
objy=datay[['Quarter', 'Year']]
numy=datay.drop(['Quarter', 'Year'],axis=1)

encode1y=pd.get_dummies(objy,drop_first = True)
encode1y.head()

datay=pd.concat([numy,encode1y],axis=1)

In [46]:
datay.head()

,checkout_price,base_price,discount amount,discount percent,compare_week_price,id,week,emailer_for_promotion,homepage_featured,num_orders,city_code,op_area,discount y/n,compare_week_price y/n,center_id_11,center_id_13,center_id_14,center_id_17,center_id_20,center_id_23,center_id_24,center_id_26,center_id_27,center_id_29,center_id_30,center_id_32,center_id_34,center_id_36,center_id_39,center_id_41,center_id_42,center_id_43,center_id_50,center_id_51,center_id_52,center_id_53,center_id_55,center_id_57,center_id_58,center_id_59,center_id_61,center_id_64,center_id_65,center_id_66,center_id_67,center_id_68,center_id_72,center_id_73,center_id_74,center_id_75,center_id_76,center_id_77,center_id_80,center_id_81,center_id_83,center_id_86,center_id_88,center_id_89,center_id_91,center_id_92,center_id_93,center_id_94,center_id_97,center_id_99,center_id_101,center_id_102,center_id_104,center_id_106,center_id_108,center_id_109,center_id_110,center_id_113,center_id_124,center_id_126,center_id_129,center_id_132,center_id_137,center_id_139,center_id_143,center_id_145,center_id_146,center_id_149,center_id_152,center_id_153,center_id_157,center_id_161,center_id_162,center_id_174,center_id_177,center_id_186,meal_id_1109,meal_id_1198,meal_id_1207,meal_id_1216,meal_id_1230,meal_id_1247,meal_id_1248,meal_id_1311,meal_id_1438,meal_id_1445,meal_id_1525,meal_id_1543,meal_id_1558,meal_id_1571,meal_id_1727,meal_id_1754,meal_id_1770,meal_id_1778,meal_id_1803,meal_id_1847,meal_id_1878,meal_id_1885,meal_id_1902,meal_id_1962,meal_id_1971,meal_id_1993,meal_id_2104,meal_id_2126,meal_id_2139,meal_id_2290,meal_id_2304,meal_id_2306,meal_id_2322,meal_id_2444,meal_id_2490,meal_id_2492,meal_id_2494,meal_id_2539,meal_id_2569,meal_id_2577,meal_id_2581,meal_id_2631,meal_id_2640,meal_id_2664,meal_id_2704,meal_id_2707,meal_id_2760,meal_id_2826,meal_id_2867,meal_id_2956,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai,city_enc_4_CH2,city_enc_4_CH3,city_enc_4_CH4,Quarter_Q2,Quarter_Q3,Quarter_Q4,Year_Y2,Year_Y3
0,-1.281184,-1.259867,-0.124498,0.399910,-0.003326,1379560,1,0,0,177,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,-1.281184,-1.362517,-0.467581,-0.562824,-0.003326,1466964,1,0,0,270,647,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,-1.294058,-1.362329,-0.425894,-0.432643,-0.003326,1346989,1,0,0,189,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0.043267,0.518969,1.596548,1.483406,-0.003326,1338232,1,0,0,54,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,-0.584095,-0.697293,-0.467581,-0.534189,-0.003326,1

##### Applying Log Transformation on the Target Feature

In [47]:
datay['num_orders']=np.log1p(datay['num_orders'])

In [48]:
train=datay[datay['week'].isin(range(1,146))]
def outliers_3(col):
    q3=round(train[col].quantile(0.75),6)
    q1=round(train[col].quantile(0.25),6)
    iqr=q3-q1
    lw = q1 - (3*iqr)
    hw = q3 + (3*iqr)  
    uo=train[train[col]>hw].shape[0]
    lo=train[train[col]<lw].shape[0]
    print('Number of Upper Outliers :',uo)
    print('Number of Lower Outliers :',lo)
    print('Percentage of Outliers :',((uo+lo)/train.shape[0])*100)

In [49]:
outliers_3('num_orders')

Number of Upper Outliers : 0
Number of Lower Outliers : 0
Percentage of Outliers : 0.0


In [50]:
datay.head()

,checkout_price,base_price,discount amount,discount percent,compare_week_price,id,week,emailer_for_promotion,homepage_featured,num_orders,city_code,op_area,discount y/n,compare_week_price y/n,center_id_11,center_id_13,center_id_14,center_id_17,center_id_20,center_id_23,center_id_24,center_id_26,center_id_27,center_id_29,center_id_30,center_id_32,center_id_34,center_id_36,center_id_39,center_id_41,center_id_42,center_id_43,center_id_50,center_id_51,center_id_52,center_id_53,center_id_55,center_id_57,center_id_58,center_id_59,center_id_61,center_id_64,center_id_65,center_id_66,center_id_67,center_id_68,center_id_72,center_id_73,center_id_74,center_id_75,center_id_76,center_id_77,center_id_80,center_id_81,center_id_83,center_id_86,center_id_88,center_id_89,center_id_91,center_id_92,center_id_93,center_id_94,center_id_97,center_id_99,center_id_101,center_id_102,center_id_104,center_id_106,center_id_108,center_id_109,center_id_110,center_id_113,center_id_124,center_id_126,center_id_129,center_id_132,center_id_137,center_id_139,center_id_143,center_id_145,center_id_146,center_id_149,center_id_152,center_id_153,center_id_157,center_id_161,center_id_162,center_id_174,center_id_177,center_id_186,meal_id_1109,meal_id_1198,meal_id_1207,meal_id_1216,meal_id_1230,meal_id_1247,meal_id_1248,meal_id_1311,meal_id_1438,meal_id_1445,meal_id_1525,meal_id_1543,meal_id_1558,meal_id_1571,meal_id_1727,meal_id_1754,meal_id_1770,meal_id_1778,meal_id_1803,meal_id_1847,meal_id_1878,meal_id_1885,meal_id_1902,meal_id_1962,meal_id_1971,meal_id_1993,meal_id_2104,meal_id_2126,meal_id_2139,meal_id_2290,meal_id_2304,meal_id_2306,meal_id_2322,meal_id_2444,meal_id_2490,meal_id_2492,meal_id_2494,meal_id_2539,meal_id_2569,meal_id_2577,meal_id_2581,meal_id_2631,meal_id_2640,meal_id_2664,meal_id_2704,meal_id_2707,meal_id_2760,meal_id_2826,meal_id_2867,meal_id_2956,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai,city_enc_4_CH2,city_enc_4_CH3,city_enc_4_CH4,Quarter_Q2,Quarter_Q3,Quarter_Q4,Year_Y2,Year_Y3
0,-1.281184,-1.259867,-0.124498,0.399910,-0.003326,1379560,1,0,0,5.181784,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,-1.281184,-1.362517,-0.467581,-0.562824,-0.003326,1466964,1,0,0,5.602119,647,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,-1.294058,-1.362329,-0.425894,-0.432643,-0.003326,1346989,1,0,0,5.247024,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0.043267,0.518969,1.596548,1.483406,-0.003326,1338232,1,0,0,4.007333,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,-0.584095,-0.697293,-0.467581,

In [51]:
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.neighbors import KNeighborsRegressor

# from sklearn.ensemble import RandomForestRegressor

# from sklearn.ensemble import BaggingRegressor
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import GradientBoostingRegressor

# from xgboost import XGBRegressor
# from sklearn.metrics import explained_variance_score

# from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

#### CatBoost Regressor

##### Model 1

In [230]:
train=datay[datay['week'].isin(range(1,146))]
test=datay[datay['week'].isin(range(146,156))]


X_train=train.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)
y_train=train['num_orders']

X_test=test.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)

CGB=CatBoostRegressor(learning_rate=0.3,loss_function='RMSE',max_depth=9,verbose=False)

CGB.fit(X_train, y_train)
CGBpred=CGB.predict(X_test)

In [231]:
Result=pd.DataFrame(CGBpred)
Result=np.expm1(Result).astype('int64')
Submission = pd.DataFrame(columns=['id', 'num_orders'])
Submission['id'] = test['id']
Submission['num_orders'] = Result.values
Submission.to_csv('Result Cat 1.csv', index=False)

### Analytics Vidhya: 53.0917287143

##### Model 2

In [53]:
train=datay[datay['week'].isin(range(1,146))]
test=datay[datay['week'].isin(range(146,156))]


X_train=train.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)
y_train=train['num_orders']

X_test=test.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)

CGB=CatBoostRegressor(learning_rate=0.1,loss_function='RMSE',max_depth=8,verbose=False)

CGB.fit(X_train, y_train)
CGBpred=CGB.predict(X_test)

In [54]:
Result=pd.DataFrame(CGBpred)
Result=np.expm1(Result).astype('int64')
Submission1 = pd.DataFrame(columns=['id', 'num_orders'])
Submission1['id'] = test['id']
Submission1['num_orders'] = Result.values
Submission1.to_csv('Submission_CatBoost.csv', index=False)

### Analytics Vidhya: 52.5823376761

In [59]:
train=datay[datay['week'].isin(range(1,146))]
test=datay[datay['week'].isin(range(146,156))]


X_train=train.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)
y_train=train['num_orders']

X_test=test.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)

CGB=CatBoostRegressor(learning_rate=0.1,loss_function='MAE',max_depth=8,verbose=False)

CGB.fit(X_train, y_train)
CGBpred=CGB.predict(X_test)

In [79]:
Result=pd.DataFrame(CGBpred)
Result=np.expm1(Result).astype('int64')
Submission1 = pd.DataFrame(columns=['id', 'num_orders'])
Submission1['id'] = test['id']
Submission1['num_orders'] = Result.values
Submission1.to_csv('Submission_CatBoost2.csv', index=False)

### Analytics Vidhya: 53.2978800680071

In [80]:
train=datay[datay['week'].isin(range(1,146))]
test=datay[datay['week'].isin(range(146,156))]


X_train=train.drop(['id','num_orders','week','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)
y_train=train['num_orders']

X_test=test.drop(['id','num_orders','week','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)

CGB=CatBoostRegressor()

In [81]:
params = {'learning_rate':[0.05,0.1,0.2],
          'max_depth': [7,8,9],
          'l2_leaf_reg':[0.1,1]
           }

In [82]:
from sklearn.model_selection import RandomizedSearchCV

In [83]:
opt = RandomizedSearchCV(CGB, params, cv=3, n_iter=100, n_jobs=1)

In [ ]:
search = opt.fit(X_train, y_train)

0:	learn: 1.1686504	total: 78.4ms	remaining: 1m 18s
1:	learn: 1.1390302	total: 149ms	remaining: 1m 14s
2:	learn: 1.1116066	total: 219ms	remaining: 1m 12s
3:	learn: 1.0862320	total: 292ms	remaining: 1m 12s
4:	learn: 1.0632224	total: 378ms	remaining: 1m 15s
5:	learn: 1.0413964	total: 450ms	remaining: 1m 14s
6:	learn: 1.0206168	total: 530ms	remaining: 1m 15s
7:	learn: 1.0015074	total: 626ms	remaining: 1m 17s
8:	learn: 0.9835217	total: 697ms	remaining: 1m 16s
9:	learn: 0.9670881	total: 778ms	remaining: 1m 17s
10:	learn: 0.9529077	total: 849ms	remaining: 1m 16s
11:	learn: 0.9384035	total: 919ms	remaining: 1m 15s
12:	learn: 0.9242142	total: 994ms	remaining: 1m 15s
13:	learn: 0.9112242	total: 1.08s	remaining: 1m 16s
14:	learn: 0.8995865	total: 1.16s	remaining: 1m 15s
15:	learn: 0.8880214	total: 1.24s	remaining: 1m 16s
16:	learn: 0.8775347	total: 1.32s	remaining: 1m 16s
17:	learn: 0.8678206	total: 1.39s	remaining: 1m 15s
18:	learn: 0.8584642	total: 1.48s	remaining: 1m 16s
19:	learn: 0.8493081	

158:	learn: 0.6018092	total: 13.3s	remaining: 1m 10s
159:	learn: 0.6014822	total: 13.3s	remaining: 1m 10s
160:	learn: 0.6009666	total: 13.4s	remaining: 1m 9s
161:	learn: 0.6004775	total: 13.5s	remaining: 1m 9s
162:	learn: 0.6001242	total: 13.6s	remaining: 1m 9s
163:	learn: 0.5996367	total: 13.6s	remaining: 1m 9s
164:	learn: 0.5990542	total: 13.7s	remaining: 1m 9s
165:	learn: 0.5986147	total: 13.8s	remaining: 1m 9s
166:	learn: 0.5981699	total: 13.9s	remaining: 1m 9s
167:	learn: 0.5977339	total: 14s	remaining: 1m 9s
168:	learn: 0.5972899	total: 14s	remaining: 1m 9s
169:	learn: 0.5968425	total: 14.1s	remaining: 1m 8s
170:	learn: 0.5963801	total: 14.2s	remaining: 1m 8s
171:	learn: 0.5960470	total: 14.3s	remaining: 1m 8s
172:	learn: 0.5956081	total: 14.3s	remaining: 1m 8s
173:	learn: 0.5951049	total: 14.4s	remaining: 1m 8s
174:	learn: 0.5945691	total: 14.5s	remaining: 1m 8s
175:	learn: 0.5941083	total: 14.5s	remaining: 1m 8s
176:	learn: 0.5935444	total: 14.6s	remaining: 1m 7s
177:	learn: 0.

317:	learn: 0.5556618	total: 25.4s	remaining: 54.5s
318:	learn: 0.5554983	total: 25.5s	remaining: 54.4s
319:	learn: 0.5552622	total: 25.5s	remaining: 54.3s
320:	learn: 0.5550577	total: 25.6s	remaining: 54.2s
321:	learn: 0.5548578	total: 25.7s	remaining: 54.1s
322:	learn: 0.5545928	total: 25.7s	remaining: 54s
323:	learn: 0.5544603	total: 25.8s	remaining: 53.8s
324:	learn: 0.5543123	total: 25.9s	remaining: 53.7s
325:	learn: 0.5540949	total: 25.9s	remaining: 53.7s
326:	learn: 0.5538928	total: 26s	remaining: 53.6s
327:	learn: 0.5537090	total: 26.1s	remaining: 53.5s
328:	learn: 0.5535125	total: 26.2s	remaining: 53.3s
329:	learn: 0.5533440	total: 26.2s	remaining: 53.2s
330:	learn: 0.5531642	total: 26.3s	remaining: 53.1s
331:	learn: 0.5529855	total: 26.4s	remaining: 53.1s
332:	learn: 0.5528146	total: 26.4s	remaining: 53s
333:	learn: 0.5526433	total: 26.5s	remaining: 52.9s
334:	learn: 0.5524431	total: 26.6s	remaining: 52.8s
335:	learn: 0.5523076	total: 26.7s	remaining: 52.7s
336:	learn: 0.5521

477:	learn: 0.5322379	total: 37.4s	remaining: 40.8s
478:	learn: 0.5321045	total: 37.5s	remaining: 40.8s
479:	learn: 0.5319907	total: 37.6s	remaining: 40.7s
480:	learn: 0.5319024	total: 37.6s	remaining: 40.6s
481:	learn: 0.5317984	total: 37.7s	remaining: 40.5s
482:	learn: 0.5316808	total: 37.8s	remaining: 40.4s
483:	learn: 0.5315340	total: 37.9s	remaining: 40.4s
484:	learn: 0.5314325	total: 37.9s	remaining: 40.3s
485:	learn: 0.5313386	total: 38s	remaining: 40.2s
486:	learn: 0.5311839	total: 38.1s	remaining: 40.1s
487:	learn: 0.5311052	total: 38.2s	remaining: 40s
488:	learn: 0.5309802	total: 38.2s	remaining: 40s
489:	learn: 0.5308928	total: 38.3s	remaining: 39.9s
490:	learn: 0.5308028	total: 38.4s	remaining: 39.8s
491:	learn: 0.5306943	total: 38.4s	remaining: 39.7s
492:	learn: 0.5305891	total: 38.5s	remaining: 39.6s
493:	learn: 0.5304900	total: 38.6s	remaining: 39.6s
494:	learn: 0.5303901	total: 38.7s	remaining: 39.5s
495:	learn: 0.5302803	total: 38.8s	remaining: 39.4s
496:	learn: 0.5301

636:	learn: 0.5177064	total: 49.5s	remaining: 28.2s
637:	learn: 0.5176391	total: 49.5s	remaining: 28.1s
638:	learn: 0.5175598	total: 49.6s	remaining: 28s
639:	learn: 0.5174597	total: 49.7s	remaining: 27.9s
640:	learn: 0.5173836	total: 49.8s	remaining: 27.9s
641:	learn: 0.5173208	total: 49.8s	remaining: 27.8s
642:	learn: 0.5172380	total: 49.9s	remaining: 27.7s
643:	learn: 0.5171593	total: 50s	remaining: 27.6s
644:	learn: 0.5170883	total: 50s	remaining: 27.5s
645:	learn: 0.5170353	total: 50.1s	remaining: 27.4s
646:	learn: 0.5169469	total: 50.2s	remaining: 27.4s
647:	learn: 0.5168758	total: 50.2s	remaining: 27.3s
648:	learn: 0.5167848	total: 50.3s	remaining: 27.2s
649:	learn: 0.5167149	total: 50.4s	remaining: 27.1s
650:	learn: 0.5166475	total: 50.5s	remaining: 27.1s
651:	learn: 0.5165741	total: 50.6s	remaining: 27s
652:	learn: 0.5165294	total: 50.6s	remaining: 26.9s
653:	learn: 0.5164670	total: 50.7s	remaining: 26.8s
654:	learn: 0.5164009	total: 50.8s	remaining: 26.7s
655:	learn: 0.516322

...

395:	learn: 0.5076404	total: 31.1s	remaining: 47.5s
396:	learn: 0.5075377	total: 31.2s	remaining: 47.4s
397:	learn: 0.5074121	total: 31.3s	remaining: 47.4s
398:	learn: 0.5073074	total: 31.4s	remaining: 47.3s
399:	learn: 0.5071986	total: 31.5s	remaining: 47.2s
400:	learn: 0.5070958	total: 31.5s	remaining: 47.1s
401:	learn: 0.5069984	total: 31.6s	remaining: 47s
402:	learn: 0.5068803	total: 31.7s	remaining: 46.9s
403:	learn: 0.5067553	total: 31.8s	remaining: 46.8s
404:	learn: 0.5066404	total: 31.8s	remaining: 46.8s
405:	learn: 0.5065393	total: 31.9s	remaining: 46.7s
406:	learn: 0.5064154	total: 32s	remaining: 46.6s
407:	learn: 0.5062827	total: 32s	remaining: 46.5s
408:	learn: 0.5061968	total: 32.1s	remaining: 46.4s
409:	learn: 0.5060895	total: 32.2s	remaining: 46.3s
410:	learn: 0.5059943	total: 32.3s	remaining: 46.2s
411:	learn: 0.5058727	total: 32.3s	remaining: 46.2s
412:	learn: 0.5057539	total: 32.4s	remaining: 46.1s
413:	learn: 0.5056279	total: 32.5s	remaining: 46s
414:	learn: 0.505531

556:	learn: 0.4938786	total: 43.2s	remaining: 34.3s
557:	learn: 0.4938108	total: 43.2s	remaining: 34.3s
558:	learn: 0.4937482	total: 43.3s	remaining: 34.2s
559:	learn: 0.4936896	total: 43.4s	remaining: 34.1s
560:	learn: 0.4936101	total: 43.5s	remaining: 34s
561:	learn: 0.4935395	total: 43.5s	remaining: 33.9s
562:	learn: 0.4934781	total: 43.6s	remaining: 33.8s
563:	learn: 0.4933922	total: 43.7s	remaining: 33.8s
564:	learn: 0.4933206	total: 43.7s	remaining: 33.7s
565:	learn: 0.4932670	total: 43.8s	remaining: 33.6s
566:	learn: 0.4931952	total: 43.9s	remaining: 33.5s
567:	learn: 0.4931377	total: 44s	remaining: 33.4s
568:	learn: 0.4930904	total: 44s	remaining: 33.3s
569:	learn: 0.4930376	total: 44.1s	remaining: 33.3s
570:	learn: 0.4929742	total: 44.2s	remaining: 33.2s
571:	learn: 0.4929076	total: 44.2s	remaining: 33.1s
572:	learn: 0.4928468	total: 44.3s	remaining: 33s
573:	learn: 0.4927931	total: 44.4s	remaining: 32.9s
574:	learn: 0.4927361	total: 44.5s	remaining: 32.9s
575:	learn: 0.492655

716:	learn: 0.4847922	total: 55.4s	remaining: 21.9s
717:	learn: 0.4847589	total: 55.5s	remaining: 21.8s
718:	learn: 0.4846975	total: 55.6s	remaining: 21.7s
719:	learn: 0.4846183	total: 55.7s	remaining: 21.7s
720:	learn: 0.4845692	total: 55.8s	remaining: 21.6s
721:	learn: 0.4845148	total: 55.9s	remaining: 21.5s
722:	learn: 0.4844616	total: 56s	remaining: 21.4s
723:	learn: 0.4844087	total: 56s	remaining: 21.4s
724:	learn: 0.4843635	total: 56.1s	remaining: 21.3s
725:	learn: 0.4843154	total: 56.3s	remaining: 21.2s
726:	learn: 0.4842739	total: 56.3s	remaining: 21.2s
727:	learn: 0.4842343	total: 56.4s	remaining: 21.1s
728:	learn: 0.4841460	total: 56.5s	remaining: 21s
729:	learn: 0.4841001	total: 56.6s	remaining: 20.9s
730:	learn: 0.4840553	total: 56.7s	remaining: 20.8s
731:	learn: 0.4839968	total: 56.7s	remaining: 20.8s
732:	learn: 0.4839541	total: 56.8s	remaining: 20.7s
733:	learn: 0.4839117	total: 56.9s	remaining: 20.6s
734:	learn: 0.4838568	total: 57s	remaining: 20.6s
735:	learn: 0.483820

876:	learn: 0.4778474	total: 1m 10s	remaining: 9.89s
877:	learn: 0.4778047	total: 1m 10s	remaining: 9.81s
878:	learn: 0.4777748	total: 1m 10s	remaining: 9.73s
879:	learn: 0.4777220	total: 1m 10s	remaining: 9.66s
880:	learn: 0.4776880	total: 1m 10s	remaining: 9.57s
881:	learn: 0.4776573	total: 1m 10s	remaining: 9.49s
882:	learn: 0.4776264	total: 1m 11s	remaining: 9.41s
883:	learn: 0.4775953	total: 1m 11s	remaining: 9.33s
884:	learn: 0.4775649	total: 1m 11s	remaining: 9.25s
885:	learn: 0.4775330	total: 1m 11s	remaining: 9.18s
886:	learn: 0.4774854	total: 1m 11s	remaining: 9.1s
887:	learn: 0.4774606	total: 1m 11s	remaining: 9.02s
888:	learn: 0.4774285	total: 1m 11s	remaining: 8.94s
889:	learn: 0.4773996	total: 1m 11s	remaining: 8.86s
890:	learn: 0.4773652	total: 1m 11s	remaining: 8.78s
891:	learn: 0.4773327	total: 1m 11s	remaining: 8.7s
892:	learn: 0.4772953	total: 1m 11s	remaining: 8.62s
893:	learn: 0.4772515	total: 1m 12s	remaining: 8.54s
894:	learn: 0.4772206	total: 1m 12s	remaining: 8

32:	learn: 0.6750297	total: 2.69s	remaining: 1m 18s
33:	learn: 0.6716848	total: 2.77s	remaining: 1m 18s
34:	learn: 0.6692159	total: 2.85s	remaining: 1m 18s
35:	learn: 0.6667965	total: 2.94s	remaining: 1m 18s
36:	learn: 0.6645130	total: 3.02s	remaining: 1m 18s
37:	learn: 0.6622522	total: 3.09s	remaining: 1m 18s
38:	learn: 0.6603175	total: 3.15s	remaining: 1m 17s
39:	learn: 0.6581630	total: 3.22s	remaining: 1m 17s
40:	learn: 0.6558049	total: 3.29s	remaining: 1m 16s
41:	learn: 0.6540986	total: 3.37s	remaining: 1m 16s
42:	learn: 0.6524576	total: 3.43s	remaining: 1m 16s
43:	learn: 0.6506757	total: 3.5s	remaining: 1m 15s
44:	learn: 0.6477837	total: 3.57s	remaining: 1m 15s
45:	learn: 0.6458440	total: 3.64s	remaining: 1m 15s
46:	learn: 0.6436450	total: 3.71s	remaining: 1m 15s
47:	learn: 0.6421132	total: 3.77s	remaining: 1m 14s
48:	learn: 0.6402114	total: 3.83s	remaining: 1m 14s
49:	learn: 0.6385152	total: 3.9s	remaining: 1m 14s
50:	learn: 0.6371399	total: 3.97s	remaining: 1m 13s
51:	learn: 0.6

In [ ]:
search.best_params_

In [74]:
Result=pd.DataFrame(CGBpred)
Result=np.expm1(Result).astype('int64')
Submission1 = pd.DataFrame(columns=['id', 'num_orders'])
Submission1['id'] = test['id']
Submission1['num_orders'] = Result.values
Submission1.to_csv('Submission_CatBoost1.csv', index=False)